## 1. Data Exploration

### Part 1 - Get the Data

In [2]:
# Establish the Working Directory
import os

# Print current working directory before change
os.getcwd()


'C:\\Users\\datam'

In [4]:
# Change the Directory
os.chdir("C:\\Users\\datam\\OneDrive\\Desktop\\python") # change as needed

# Verify the change
os.getcwd()

'C:\\Users\\datam\\OneDrive\\Desktop\\python'

In [8]:
# missingno is not part of anaconda
# pip install missingno

In [7]:
# Required Python libraries (install using pip if needed)
# pip install pandas numpy scipy scikit-learn missingno

import pandas as pd
import numpy as np
from scipy.stats import describe
import missingno as msno
from sklearn.feature_selection import VarianceThreshold

In [9]:
# Reading data
gettysburg = pd.read_csv('gettysburg.csv')

# Check column names and dimensions
gettysburg.columns

Index(['type', 'state', 'regiment_or_battery', 'brigade', 'division', 'corps',
       'army', 'july1_Commander', 'Cdr_casualty', 'men', 'killed', 'wounded',
       'captured', 'missing', 'total_casualties', '3inch_rifles',
       '4.5inch_rifles', '10lb_parrots', '12lb_howitzers', '12lb_napoleons',
       '6lb_howitzers', '24lb_howitzers', '20lb_parrots', '12lb_whitworths',
       '14lb_rifles', 'total_guns'],
      dtype='object')

In [10]:
gettysburg.shape

(590, 26)

In [11]:
gettysburg.head

<bound method NDFrame.head of           type      state regiment_or_battery        brigade   division  \
0     Infantry    Indiana                19th       Meredith  Wadsworth   
1     Infantry   Michigan                24th       Meredith  Wadsworth   
2     Infantry  Wisconsin                 2nd       Meredith  Wadsworth   
3     Infantry  Wisconsin                 6th       Meredith  Wadsworth   
4     Infantry  Wisconsin                 7th       Meredith  Wadsworth   
..         ...        ...                 ...            ...        ...   
585  Artillery   Virginia    Battery Breathed        Beckham     Stuart   
586  Artillery   Virginia    Battery McGregor        Beckham     Stuart   
587    Cavalry    Georgia    Phillips  Legion        Hampton     Stuart   
588    Cavalry   Virginia                 1st  Fitzhugh  Lee     Stuart   
589    Cavalry   Virginia                 2nd  Fitzhugh  Lee     Stuart   

        corps         army       july1_Commander      Cdr_casualty  m

### Part 2 - Duplicates

In [12]:
dupes = gettysburg.duplicated()
dupes.value_counts()

False    587
True       3
Name: count, dtype: int64

In [15]:
# Identify the duplicate values
dupes[dupes].index

Index([587, 588, 589], dtype='int64')

In [16]:
# Remove the duplicates
gettysburg = gettysburg.drop_duplicates(keep='first')

### Part 3 - Understanding the Data

In [19]:
# Filter and descriptive stats
descr_stats = gettysburg[(gettysburg['army'] == 'Confederate') & 
                        (gettysburg['type'] == 'Infantry')].describe()
descr_stats

,men,killed,wounded,captured,missing,total_casualties,3inch_rifles,4.5inch_rifles,10lb_parrots,12lb_howitzers,12lb_napoleons,6lb_howitzers,24lb_howitzers,20lb_parrots,12lb_whitworths,14lb_rifles,total_guns
count,171.000000,171.000000,171.000000,170.0,160.000000,170.000000,171.0,171.0,171.0,171.0,171.0,171.0,171.0,171.0,171.0,171.0,171.0
mean,335.263158,26.269006,69.532164,0.0,34.156250,127.341176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,104.007894,22.785396,52.855440,0.0,30.249393,88.296241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,135.000000,0.000000,3.000000,0.0,0.000000,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,267.000000,11.000000,35.000000,0.0,11.000000,65.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,316.000000,19.000000,61.000000,0.0,26.000000,106.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,380.500000,37.000000,91.500000,0.0,49.250000,177.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,843.000000,172.000000,443.000000,0.0,158.000000,687.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Count distinct values for a column in this case 'type'
gettysburg['type'].nunique()

3

In [21]:
# Get categorical columns
gettysburg_cat = gettysburg.select_dtypes(include=['object'])

# Count unique values for all categorical columns
gettysburg_cat.nunique()

type                     3
state                   30
regiment_or_battery    276
brigade                124
division                38
corps                   14
army                     2
july1_Commander        586
Cdr_casualty             6
dtype: int64

In [22]:
# Group by and count the values in that group
gettysburg_cat.groupby('Cdr_casualty').size()

Cdr_casualty
captured              6
killed               29
mortally wounded     24
no                  405
wounded             104
wounded-captured     19
dtype: int64

In [23]:
# Pandas crosstabs
pd.crosstab(gettysburg_cat['army'], gettysburg_cat['Cdr_casualty'])

Cdr_casualty,captured,killed,mortally wounded,no,wounded,wounded-captured
army,,,,,,
Confederate,2,15,13,165,44,17
Union,4,14,11,240,60,2


In [25]:
# Missing value exploration -----------------------------------------------
na_count = gettysburg.isna().sum()
#na_count
na_df = pd.DataFrame(na_count, columns=['na_count'])
na_df

,na_count
type,0
state,0
regiment_or_battery,0
brigade,0
division,0
corps,0
army,0
july1_Commander,0
Cdr_casualty,0
men,0


In [26]:
# Find indices of NA in 'killed' column
gettysburg[gettysburg['killed'].isna()].index

Index([393, 529, 530, 553, 574, 578], dtype='int64')

In [29]:
# Code missing values for number of missing soldiers
# gettysburg['missing_isNA'] = np.where(gettysburg['missing'].isna(), 1, 0) # throws error

# Use .loc for safe assignment
gettysburg.loc[:, 'missing_isNA'] = np.where(gettysburg['missing'].isna(), 1, 0) # creates the dummy variable
gettysburg.loc[:, 'missing'] = gettysburg['missing'].fillna(0) # turns those missing values to 0

In [30]:
# Simple value counts to validate
gettysburg['missing_isNA'].value_counts()

missing_isNA
0    570
1     17
Name: count, dtype: int64

In [32]:
# Low or no variance ------------------------------------------------------
# Using VarianceThreshold from sklearn
selector = VarianceThreshold(threshold=0)  # 0 variance features
selector.fit(gettysburg.select_dtypes(include=['int64', 'float64']))
feature_variance = pd.DataFrame({
    'feature': gettysburg.select_dtypes(include=['int64', 'float64']).columns,
    'zeroVar': ~selector.get_support()
})
feature_variance

,feature,zeroVar
0,men,False
1,killed,False
2,wounded,False
3,captured,False
4,missing,False
5,total_casualties,False
6,3inch_rifles,False
7,4.5inch_rifles,True
8,10lb_parrots,False
9,12lb_howitzers,False


In [36]:
# Filter out zero variance features

# Filter numeric columns with non-zero variance
non_zero_var_cols = feature_variance[feature_variance['zeroVar'] == False]['feature']

# Combine with non-numeric columns
non_numeric_cols = gettysburg.select_dtypes(exclude=['int64', 'float64']).columns
gettysburg_fltrd = gettysburg[list(non_zero_var_cols) + list(non_numeric_cols)]
gettysburg_fltrd.shape

(587, 26)

### Part 4 - Treatment of the Dataframe

In [37]:
gettysburg_treated = pd.get_dummies(gettysburg_fltrd, 
                                  dummy_na=True,
                                  drop_first=True)
gettysburg_treated.head()

,men,killed,wounded,captured,missing,total_casualties,3inch_rifles,10lb_parrots,12lb_howitzers,12lb_napoleons,...,july1_Commander_MAJ William Parsley,july1_Commander_MAJ William Terry,july1_Commander_Maj Thomas Jefferson Lipscomb,july1_Commander_nan,Cdr_casualty_killed,Cdr_casualty_mortally wounded,Cdr_casualty_no,Cdr_casualty_wounded,Cdr_casualty_wounded-captured,Cdr_casualty_nan
0,308,33.0,126.0,2.0,52.0,213.0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
1,496,96.0,186.0,13.0,76.0,371.0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,False
2,302,45.0,135.0,18.0,42.0,240.0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,False
3,344,39.0,110.0,3.0,22.0,174.0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
4,364,37.0,102.0,3.0,43.0,185.0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False


In [40]:
# The dataframe is a mess and needs cleaning
bool_cols = gettysburg_treated.select_dtypes(include=['bool']).columns
gettysburg_treated[bool_cols] = gettysburg_treated[bool_cols].astype(int) # convert type

In [41]:
# Now, remove low-variance dummy variables
# Identify dummy columns (those created by get_dummies, typically have '_' in name)
# However, if you use snake case column headers this could be an issue and this can be optimized
# Thus, it is better to use pyvtreat
dummy_cols = [col for col in gettysburg_treated.columns if '_' in col]
#print("Dummy columns before filtering:", dummy_cols)

# Apply VarianceThreshold to dummy columns
selector = VarianceThreshold(threshold=0.05)  # Adjust threshold as needed (0.01 means 1% variance)
selector.fit(gettysburg_treated[dummy_cols])

# Create a mask for dummy columns with sufficient variance
dummy_variance = pd.DataFrame({
    'feature': dummy_cols,
    'keep': selector.get_support()
})
high_var_dummy_cols = dummy_variance[dummy_variance['keep']]['feature']

# Combine high-variance dummy columns with non-dummy columns
non_dummy_cols = [col for col in gettysburg_treated.columns if col not in dummy_cols]
gettysburg_cleaned = gettysburg_treated[list(high_var_dummy_cols) + non_dummy_cols]
gettysburg_cleaned.shape

(587, 34)

In [42]:
gettysburg_cleaned.head()

,total_casualties,3inch_rifles,10lb_parrots,12lb_howitzers,12lb_napoleons,20lb_parrots,total_guns,type_Cavalry,type_Infantry,state_Georgia,...,corps_Slocum,corps_Sykes,army_Union,Cdr_casualty_no,Cdr_casualty_wounded,men,killed,wounded,captured,missing
0,213.0,0,0,0,0,0,0,0,1,0,...,0,0,1,1,0,308,33.0,126.0,2.0,52.0
1,371.0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,496,96.0,186.0,13.0,76.0
2,240.0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,302,45.0,135.0,18.0,42.0
3,174.0,0,0,0,0,0,0,0,1,0,...,0,0,1,1,0,344,39.0,110.0,3.0,22.0
4,185.0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,364,37.0,102.0,3.0,43.0


In [43]:
gettysburg_cleaned.columns

Index(['total_casualties', '3inch_rifles', '10lb_parrots', '12lb_howitzers',
       '12lb_napoleons', '20lb_parrots', 'total_guns', 'type_Cavalry',
       'type_Infantry', 'state_Georgia', 'state_New York',
       'state_North Carolina', 'state_Pennsylvania', 'state_US',
       'state_Virginia', 'division_Artillery_Bde', 'corps_Hancock',
       'corps_Hill', 'corps_Howard', 'corps_Longstreet', 'corps_Pleasonton',
       'corps_Reynolds', 'corps_Sedgwick', 'corps_Sickles', 'corps_Slocum',
       'corps_Sykes', 'army_Union', 'Cdr_casualty_no', 'Cdr_casualty_wounded',
       'men', 'killed', 'wounded', 'captured', 'missing'],
      dtype='object')

### Part 5 Correlation

In [47]:
# Find highly correlated features
# Adjust the upper_tri value as needed
df_corr = gettysburg_cleaned.corr(method='spearman') # pearson is default

correlation_matrix = df_corr.abs()
upper_tri = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
high_corr = [column for column in upper_tri.columns 
            if any(upper_tri[column] > 0.9)]

high_corr

['killed', 'wounded']

In [ ]:
# gettysburg_noHighCorr = gettysburg_treated.drop(columns=high_corr)

### End